# Code Analyzer - Eating Our Own Dog Food

Intelligent Code Analyzer built on Fractal Manifold, analyzing its own codebase.

## Architecture: Double Loop (Action + Reflection)

```text
┌─────────────────────────────────────────────────────────────────────────┐
│                    CYBERNETIC DOUBLE LOOP                               │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  ╔═══════════════════ LOOP 1: ACTION ════════════════════╗              │
│  ║                                                       ║              │
│  ║  SENSE              PROCESS              ACT          ║              │
│  ║  ─────              ───────              ───          ║              │
│  ║  Perceptron ──────► Pipeline ──────────► Actuator ────╫──► OUTPUT    │
│  ║  (input)            (HLLSet→HRT)         (response)   ║              │
│  ║                                                       ║              │
│  ╚═══════════════════════════════════════════════════════╝              │
│                                                    │                    │
│                                           (own output)                  │
│                                                    │                    │
│  ╔═══════════════════ LOOP 2: REFLECTION ════════════════╗              │
│  ║                                                       ║              │
│  ║  OBSERVE            ENCODE               COMMIT       ║              │
│  ║  ───────            ──────               ──────       ║              │
│  ║  Response ────────► HLLSet ────────────► Memory ◄─────╫──┘           │
│  ║  (self-observe)     (self-encode)        (manifold)   ║              │
│  ║                                                       ║              │
│  ╚═══════════════════════════════════════════════════════╝              │
│                                                                         │
│  This is CYBERNETIC AI:                                                 │
│    - System observes its own outputs                                    │
│    - Encodes them into memory                                           │
│    - Future behavior shaped by past behavior                            │
│    - Different from current AI (stateless, no self-reflection)          │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

**Loop 1 (Action)**: Perceptron → Pipeline → Actuator → Output
**Loop 2 (Reflection)**: Output → HLLSet → Commit → Memory

The reflection loop makes this **cybernetic** - the system has memory of its own actions.

## 1. Imports and Setup

In [1]:
import os
import json
import time
import warnings
from pathlib import Path
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Callable, Iterator
from abc import ABC, abstractmethod

# Suppress GPU warnings
os.environ.setdefault("CUDA_VISIBLE_DEVICES", "0")
warnings.filterwarnings("ignore", message=".*cuda capability.*")

# Core imports
from core import (
    SparseHRT3D,
    Sparse3DConfig,
    SparseAM3D,
    SparseLattice3D,
    Edge3D,
    HLLSet,
    get_device,
    __version__
)

# Manifold Algebra
from core.manifold_algebra import (
    UniversalID,
    LookupTable,
    START, END,
    ProcessingResult,
    unified_process,
    build_w_from_am,
    input_to_hllset,
    build_sub_hrt,
    merge_hrt,
    Sparse3DMatrix,
    project_layer,
    reachable_from,
)

print(f"Fractal Manifold Code Analyzer")
print(f"Core v{__version__}")
print(f"Device: {get_device()}")

Fractal Manifold Code Analyzer
Core v0.7.0
Device: cuda


## 2. Configuration

In [2]:
# System configuration
N_GRAM_SIZE = 3
P_BITS = 10
H_BITS = 32

config = Sparse3DConfig(
    p_bits=P_BITS,
    h_bits=H_BITS,
    max_n=N_GRAM_SIZE
)

# Project root
PROJECT_ROOT = Path(".").resolve()

print(f"=== Configuration ===")
print(f"Project root: {PROJECT_ROOT}")
print(f"N-gram size: {N_GRAM_SIZE}")
print(f"AM dimension: {config.dimension:,}")

=== Configuration ===
Project root: /home/alexmy/SGS/SGS_lib/fractal_manifold/fractal_manifold
N-gram size: 3
AM dimension: 32,770


## 3. Commit Store

In [3]:
@dataclass
class Commit:
    """A committed HRT state."""
    id: str                    # Commit hash (content-addressed)
    hrt: SparseHRT3D          # The HRT state
    W: Dict                   # Transition matrix
    source: str               # Source file path
    file_type: str            # File type (py, md, etc.)
    timestamp: float          # When committed
    parent_id: Optional[str]  # Previous commit
    
    @property
    def summary(self) -> str:
        return f"Commit({self.id[:8]}): {self.source} [{self.hrt.nnz} edges]"


@dataclass
class CommitStore:
    """Store for committed HRT states."""
    commits: Dict[str, Commit] = field(default_factory=dict)
    head: Optional[str] = None  # Current HEAD commit
    
    def commit(self, hrt: SparseHRT3D, W: Dict, source: str, file_type: str) -> Commit:
        """Commit an HRT state."""
        # Content-addressed ID from AM hash
        commit_id = hrt.am.name[:16]
        
        c = Commit(
            id=commit_id,
            hrt=hrt,
            W=W,
            source=source,
            file_type=file_type,
            timestamp=time.time(),
            parent_id=self.head
        )
        
        self.commits[commit_id] = c
        self.head = commit_id
        return c
    
    def get_head(self) -> Optional[Commit]:
        """Get current HEAD commit."""
        return self.commits.get(self.head) if self.head else None
    
    def history(self) -> List[Commit]:
        """Get commit history from HEAD."""
        result = []
        current = self.head
        while current:
            c = self.commits.get(current)
            if c:
                result.append(c)
                current = c.parent_id
            else:
                break
        return result


# Initialize commit store
store = CommitStore()
print(f"Commit store initialized")

Commit store initialized


## 4. Perceptron Base Class

In [4]:
class Perceptron(ABC):
    """
    Base class for file-type perceptrons.
    
    Each perceptron:
    1. Finds files of its type
    2. Extracts text content
    3. Processes via unified pipeline
    4. Commits after each file
    """
    
    def __init__(self, name: str, extensions: List[str], config: Sparse3DConfig):
        self.name = name
        self.extensions = extensions
        self.config = config
        self.lut = LookupTable(config=config)
        self.lut.add_ntoken(START)
        self.lut.add_ntoken(END)
        self.files_processed = 0
        self.total_tokens = 0
    
    def find_files(self, root: Path, exclude_dirs: set = None) -> Iterator[Path]:
        """Find all files matching extensions."""
        exclude_dirs = exclude_dirs or {'__pycache__', '.git', 'build', '.ipynb_checkpoints', 'deprecated'}
        
        for path in root.rglob('*'):
            if path.is_file() and path.suffix in self.extensions:
                # Skip excluded directories
                if not any(ex in path.parts for ex in exclude_dirs):
                    yield path
    
    @abstractmethod
    def extract_text(self, path: Path) -> str:
        """Extract text content from file."""
        pass
    
    def process_file(
        self, 
        path: Path, 
        current_hrt: SparseHRT3D, 
        current_W: Dict,
        store: CommitStore
    ) -> tuple[SparseHRT3D, Dict, Commit]:
        """Process a single file and commit."""
        # Extract text
        text = self.extract_text(path)
        if not text.strip():
            return current_hrt, current_W, None
        
        # Unified processing
        result = unified_process(
            text, 
            current_hrt, 
            current_W, 
            self.config, 
            self.lut,
            N_GRAM_SIZE
        )
        
        # Update state
        new_hrt = result.merged_hrt
        new_W = build_w_from_am(new_hrt.am, self.config)
        
        # Commit
        relative_path = str(path.relative_to(PROJECT_ROOT))
        commit = store.commit(new_hrt, new_W, relative_path, self.name)
        
        self.files_processed += 1
        self.total_tokens += len(self.lut.ntoken_to_index)
        
        return new_hrt, new_W, commit
    
    def process_all(
        self,
        root: Path,
        current_hrt: SparseHRT3D,
        current_W: Dict,
        store: CommitStore,
        verbose: bool = True,
        max_files: int = None
    ) -> tuple[SparseHRT3D, Dict]:
        """
        Process files of this type.
        
        Args:
            max_files: Limit number of files to process (None = all)
        """
        files = list(self.find_files(root))
        
        # Limit if specified
        if max_files is not None:
            files = files[:max_files]
        
        if verbose:
            total_found = len(list(self.find_files(root))) if max_files else len(files)
            print(f"\n[{self.name}] Processing {len(files)}/{total_found} files")
        
        for path in files:
            try:
                current_hrt, current_W, commit = self.process_file(
                    path, current_hrt, current_W, store
                )
                if verbose and commit:
                    print(f"  ✓ {commit.source} [{current_hrt.nnz} edges]")
            except Exception as e:
                if verbose:
                    print(f"  ✗ {path}: {e}")
        
        return current_hrt, current_W


print("Perceptron base class defined")
print("  max_files parameter added to process_all()")

Perceptron base class defined
  max_files parameter added to process_all()


## 5. File-Type Perceptrons

In [5]:
class PythonPerceptron(Perceptron):
    """Perceptron for .py files."""
    
    def __init__(self, config: Sparse3DConfig):
        super().__init__("p_py", [".py"], config)
    
    def extract_text(self, path: Path) -> str:
        """Extract Python source code."""
        with open(path, 'r', encoding='utf-8', errors='ignore') as f:
            return f.read()


class NotebookPerceptron(Perceptron):
    """Perceptron for .ipynb files."""
    
    def __init__(self, config: Sparse3DConfig):
        super().__init__("p_nb", [".ipynb"], config)
    
    def extract_text(self, path: Path) -> str:
        """Extract code and markdown from notebook cells."""
        with open(path, 'r', encoding='utf-8', errors='ignore') as f:
            nb = json.load(f)
        
        texts = []
        for cell in nb.get('cells', []):
            source = cell.get('source', [])
            if isinstance(source, list):
                texts.append(''.join(source))
            else:
                texts.append(source)
        
        return '\n\n'.join(texts)


class MarkdownPerceptron(Perceptron):
    """Perceptron for .md files."""
    
    def __init__(self, config: Sparse3DConfig):
        super().__init__("p_md", [".md"], config)
    
    def extract_text(self, path: Path) -> str:
        """Extract markdown text."""
        with open(path, 'r', encoding='utf-8', errors='ignore') as f:
            return f.read()


class CPerceptron(Perceptron):
    """Perceptron for .c and .pyx (Cython) files."""
    
    def __init__(self, config: Sparse3DConfig):
        super().__init__("p_c", [".c", ".pyx", ".h"], config)
    
    def extract_text(self, path: Path) -> str:
        """Extract C/Cython source code."""
        with open(path, 'r', encoding='utf-8', errors='ignore') as f:
            return f.read()


class PDFPerceptron(Perceptron):
    """Perceptron for .pdf files (placeholder)."""
    
    def __init__(self, config: Sparse3DConfig):
        super().__init__("p_pdf", [".pdf"], config)
    
    def extract_text(self, path: Path) -> str:
        """Extract text from PDF (requires pypdf or similar)."""
        try:
            from pypdf import PdfReader
            reader = PdfReader(path)
            texts = [page.extract_text() or '' for page in reader.pages]
            return '\n'.join(texts)
        except ImportError:
            # Skip if pypdf not installed
            return ""
        except Exception:
            return ""


# Create perceptrons
perceptrons = {
    'py': PythonPerceptron(config),
    'nb': NotebookPerceptron(config),
    'md': MarkdownPerceptron(config),
    'c': CPerceptron(config),
    'pdf': PDFPerceptron(config),
}

print(f"Created {len(perceptrons)} perceptrons:")
for name, p in perceptrons.items():
    print(f"  {p.name}: {p.extensions}")

Created 5 perceptrons:
  p_py: ['.py']
  p_nb: ['.ipynb']
  p_md: ['.md']
  p_c: ['.c', '.pyx', '.h']
  p_pdf: ['.pdf']


In [6]:
class PromptPerceptron(Perceptron):
    """
    Perceptron for user prompts/queries.
    
    Treats user input exactly like file input:
    - Goes through unified pipeline
    - Gets committed
    - Contributes to manifold (learning from queries!)
    """
    
    def __init__(self, config: Sparse3DConfig):
        super().__init__("p_prompt", [], config)  # No file extensions
        self.prompt_history: List[str] = []
    
    def extract_text(self, path: Path) -> str:
        """Not used - prompts come directly as text."""
        return ""
    
    def process_prompt(
        self,
        prompt: str,
        current_hrt: SparseHRT3D,
        current_W: Dict,
        store: CommitStore
    ) -> tuple[SparseHRT3D, Dict, Commit, ProcessingResult]:
        """
        Process a user prompt and commit.
        
        Returns updated HRT, W, commit, and processing result for query use.
        """
        if not prompt.strip():
            return current_hrt, current_W, None, None
        
        # Track history
        self.prompt_history.append(prompt)
        
        # Unified processing (same as files!)
        result = unified_process(
            prompt,
            current_hrt,
            current_W,
            self.config,
            self.lut,
            N_GRAM_SIZE
        )
        
        # Update state
        new_hrt = result.merged_hrt
        new_W = build_w_from_am(new_hrt.am, self.config)
        
        # Commit with prompt as source
        prompt_id = f"prompt_{len(self.prompt_history)}"
        commit = store.commit(new_hrt, new_W, prompt_id, self.name)
        
        self.files_processed += 1
        
        return new_hrt, new_W, commit, result


# Create prompt perceptron (LUT will be assigned in Initialize Empty HRT cell)
prompt_perceptron = PromptPerceptron(config)

print("PromptPerceptron class defined")
print("  - Processes user queries through unified pipeline")
print("  - Commits each query (manifold learns from interactions)")
print("  - Returns processing result for retrieval")

PromptPerceptron class defined
  - Processes user queries through unified pipeline
  - Commits each query (manifold learns from interactions)
  - Returns processing result for retrieval


## 6. Actuators (Output Actions)

In [7]:
from abc import ABC, abstractmethod
from datetime import datetime


class Actuator(ABC):
    """
    Base class for actuators - turn processed data into action.
    
    Completes the sense-process-act loop:
        Perceptron (sense) → Pipeline (process) → Actuator (act)
    
    Key insight: Actuator output can feed back into the manifold!
    """
    
    def __init__(self, name: str):
        self.name = name
        self.actions_taken = 0
    
    @abstractmethod
    def act(self, commit: Commit, result: ProcessingResult, **kwargs) -> str:
        """
        Perform action based on processed result.
        
        Returns action summary string.
        """
        pass


class LogActuator(Actuator):
    """
    Actuator for file perceptrons - logs ingestion to file.
    """
    
    def __init__(self, log_path: Path = None):
        super().__init__("a_log")
        self.log_path = log_path or Path("ingestion.log")
        self.entries: List[str] = []
    
    def act(self, commit: Commit, result: ProcessingResult, **kwargs) -> str:
        """Log the ingestion event."""
        timestamp = datetime.fromtimestamp(commit.timestamp).isoformat()
        entry = f"[{timestamp}] {commit.file_type}: {commit.source} | edges={commit.hrt.nnz} | commit={commit.id[:8]}\n"
        
        self.entries.append(entry)
        self.actions_taken += 1
        
        # Write to file
        with open(self.log_path, 'a') as f:
            f.write(entry)
        
        return f"Logged: {commit.source}"
    
    def flush(self) -> None:
        """Write all buffered entries to log file."""
        with open(self.log_path, 'w') as f:
            f.writelines(self.entries)


class ResponseActuator(Actuator):
    """
    Actuator for prompt perceptron - generates query response.
    
    FEEDBACK LOOP: The response itself is ingested back into the manifold!
    This creates co-adaptive learning:
        Query → Response → HLLSet → Commit → (shapes future responses)
    """
    
    def __init__(self):
        super().__init__("a_response")
        self.responses: List[Dict] = []
    
    def act(
        self, 
        commit: Commit, 
        result: ProcessingResult, 
        query_results: List[tuple] = None,
        hrt: SparseHRT3D = None,
        W: Dict = None,
        store: CommitStore = None,
        lut: LookupTable = None,
        config: Sparse3DConfig = None,
        ingest_response: bool = True,
        **kwargs
    ) -> tuple[str, SparseHRT3D, Dict]:
        """
        Generate response and optionally ingest it back.
        
        Returns:
            (response_text, updated_hrt, updated_W)
        """
        
        # Build response text
        lines = [
            f"Query: {commit.source}",
            f"Commit: {commit.id[:8]}",
            f"Results ({len(query_results or [])} found):",
        ]
        
        for i, (ntoken, score) in enumerate(query_results or [], 1):
            lines.append(f"  {i:2d}. [{score:5.1f}] {ntoken}")
        
        response_text = "\n".join(lines)
        
        # Track response
        response_record = {
            "timestamp": datetime.fromtimestamp(commit.timestamp).isoformat(),
            "prompt": commit.source,
            "commit_id": commit.id[:8],
            "response": response_text,
            "ingested": False,
        }
        
        new_hrt = hrt
        new_W = W
        
        # FEEDBACK LOOP: Ingest response back into manifold
        if ingest_response and hrt and store and lut and config:
            response_result = unified_process(
                response_text,
                hrt,
                W,
                config,
                lut,
                N_GRAM_SIZE
            )
            
            new_hrt = response_result.merged_hrt
            new_W = build_w_from_am(new_hrt.am, config)
            
            # Commit response as its own entry
            response_id = f"response_{len(self.responses) + 1}"
            store.commit(new_hrt, new_W, response_id, "a_response")
            
            response_record["ingested"] = True
            response_record["response_commit"] = response_id
        
        self.responses.append(response_record)
        self.actions_taken += 1
        
        return response_text, new_hrt, new_W
    
    def history(self) -> List[Dict]:
        """Get response history."""
        return self.responses


class CompositeActuator(Actuator):
    """
    Actuator that chains multiple actuators.
    """
    
    def __init__(self, actuators: List[Actuator]):
        super().__init__("a_composite")
        self.actuators = actuators
    
    def act(self, commit: Commit, result: ProcessingResult, **kwargs) -> str:
        """Run all child actuators."""
        outputs = []
        for actuator in self.actuators:
            output = actuator.act(commit, result, **kwargs)
            if isinstance(output, tuple):
                output = output[0]  # Handle ResponseActuator tuple
            outputs.append(f"[{actuator.name}] {output}")
            self.actions_taken += 1
        return "\n".join(outputs)


# Create actuators
log_actuator = LogActuator(PROJECT_ROOT / "data" / "ingestion.log")
response_actuator = ResponseActuator()

print("Actuators created:")
print(f"  {log_actuator.name}: Logs file ingestion to {log_actuator.log_path}")
print(f"  {response_actuator.name}: Generates query responses AND ingests them back!")
print()
print("Feedback loop:")
print("  Query → HLLSet → Commit → Response → HLLSet → Commit")
print("  (manifold learns from both questions AND answers)")

Actuators created:
  a_log: Logs file ingestion to /home/alexmy/SGS/SGS_lib/fractal_manifold/fractal_manifold/data/ingestion.log
  a_response: Generates query responses AND ingests them back!

Feedback loop:
  Query → HLLSet → Commit → Response → HLLSet → Commit
  (manifold learns from both questions AND answers)


## 6. Initialize Empty HRT

In [8]:
# Create shared LUT (all perceptrons contribute)
shared_lut = LookupTable(config=config)
shared_lut.add_ntoken(START)
shared_lut.add_ntoken(END)

# Update all perceptrons to use shared LUT
for p in perceptrons.values():
    p.lut = shared_lut

# Also set prompt perceptron's LUT
prompt_perceptron.lut = shared_lut

# Initialize empty HRT
empty_am = SparseAM3D.from_edges(config, [])
empty_lattice = SparseLattice3D.from_sparse_am(empty_am)
current_hrt = SparseHRT3D(
    am=empty_am, 
    lattice=empty_lattice, 
    config=config, 
    lut=frozenset(), 
    step=0
)
current_W = {}

print(f"Initial HRT: {current_hrt.nnz} edges")
print(f"Shared LUT: {len(shared_lut.ntoken_to_index)} entries")
print(f"Prompt perceptron ready")

Initial HRT: 0 edges
Shared LUT: 2 entries
Prompt perceptron ready


## 7. Run All Perceptrons

In [9]:
# ═══════════════════════════════════════════════════════════
# CONFIGURATION: Limit files per type for faster testing
# Set to None for full ingestion
# ═══════════════════════════════════════════════════════════
MAX_FILES_PER_TYPE = 8  # Change to None for all files

print("═" * 60)
print("INGESTING PROJECT FILES")
print(f"Max files per type: {MAX_FILES_PER_TYPE or 'ALL'}")
print("═" * 60)

start_time = time.time()

# Process in order: Python first (core), then docs, then notebooks
processing_order = ['py', 'c', 'md', 'nb', 'pdf']

for ptype in processing_order:
    p = perceptrons[ptype]
    current_hrt, current_W = p.process_all(
        PROJECT_ROOT,
        current_hrt,
        current_W,
        store,
        verbose=True,
        max_files=MAX_FILES_PER_TYPE
    )

total_time = time.time() - start_time

print()
print("═" * 60)
print("INGESTION COMPLETE")
print("═" * 60)
print(f"Total time: {total_time:.2f}s")
print(f"Total commits: {len(store.commits)}")
print(f"Total edges: {current_hrt.nnz:,}")
print(f"Total n-tokens: {len(shared_lut.ntoken_to_index):,}")

════════════════════════════════════════════════════════════
INGESTING PROJECT FILES
Max files per type: 8
════════════════════════════════════════════════════════════

[p_py] Processing 8/9670 files
  ✓ main.py [28 edges]
  ✓ setup.py [130 edges]
  ✓ core/__init__.py [2543 edges]
  ✓ core/algebra.py [10269 edges]
  ✓ core/constants.py [11028 edges]
  ✓ core/db_ingestion.py [13782 edges]
  ✓ core/disambiguation.py [15056 edges]
  ✓ core/graph_visualizer.py [19929 edges]

[p_c] Processing 8/9827 files
  ✓ core/hll_core.pyx [25791 edges]
  ✓ core/hll_core.c [169565 edges]
  ✓ .venv/lib/python3.13/site-packages/cuda/ccuda.pyx [169644 edges]
  ✓ .venv/lib/python3.13/site-packages/cuda/ccudart.pyx [169669 edges]
  ✓ .venv/lib/python3.13/site-packages/cuda/cnvrtc.pyx [169694 edges]
  ✓ .venv/lib/python3.13/site-packages/cuda/cuda.pyx [169846 edges]
  ✓ .venv/lib/python3.13/site-packages/cuda/cudart.pyx [169884 edges]
  ✓ .venv/lib/python3.13/site-packages/cuda/nvrtc.pyx [169922 edges]

[p_md

## 8. Manifold Statistics

In [10]:
print("═" * 60)
print("MANIFOLD STATISTICS")
print("═" * 60)
print()

# Layer statistics
print("Layer breakdown:")
AM = Sparse3DMatrix.from_am(current_hrt.am, config)
for n in range(config.max_n):
    layer = project_layer(AM, n)
    print(f"  Layer {n} ({n+1}-grams): {layer.nnz:,} edges")

print()

# Perceptron statistics
print("Per-perceptron breakdown:")
for ptype in processing_order:
    p = perceptrons[ptype]
    print(f"  {p.name}: {p.files_processed} files")

print()

# Commit history
print("Recent commits:")
for commit in store.history()[:5]:
    print(f"  {commit.summary}")

════════════════════════════════════════════════════════════
MANIFOLD STATISTICS
════════════════════════════════════════════════════════════

Layer breakdown:
  Layer 0 (1-grams): 91,356 edges
  Layer 1 (2-grams): 68,618 edges
  Layer 2 (3-grams): 97,351 edges

Per-perceptron breakdown:
  p_py: 8 files
  p_c: 8 files
  p_md: 8 files
  p_nb: 7 files
  p_pdf: 0 files

Recent commits:
  Commit(00ae26e3): 6_code_analyzer.ipynb [257325 edges]
  Commit(3c37bda8): 5_manifold_algebra.ipynb [249669 edges]
  Commit(ba4ff0a5): 4_sheaf_retrieval_simulation.ipynb [248001 edges]
  Commit(764bc166): 3_sparse_hrt_3d.ipynb [227893 edges]
  Commit(8c0cb590): 2_sparse_hrt.ipynb [221432 edges]


## 9. Query the Manifold

In [13]:
def query(text: str, top_k: int = 10) -> List[tuple]:
    """
    Query the manifold for related n-tokens.
    
    Returns top-k n-tokens by connectivity to query.
    """
    # Process query through unified pipeline
    result = unified_process(
        text,
        current_hrt,
        current_W,
        config,
        shared_lut,
        N_GRAM_SIZE
    )
    
    # Get indices from query HLLSet
    query_indices = set()
    for edge in result.context_edges:
        query_indices.add(edge.row)
        query_indices.add(edge.col)
    
    # Build AM from current HRT
    AM = Sparse3DMatrix.from_am(current_hrt.am, config)
    
    # Find reachable from query (1-hop neighbors)
    layer0 = project_layer(AM, 0)
    reachable = reachable_from(layer0, query_indices, hops=1)
    
    # Convert layer to dict for scoring
    layer0_dict = layer0.to_dict()
    
    # Score by how many query nodes connect to each
    scores = {}
    for idx in reachable:
        if idx in layer0_dict:
            scores[idx] = sum(layer0_dict[idx].values())
    
    # Get top-k
    top = sorted(scores.items(), key=lambda x: -x[1])[:top_k]
    
    # Resolve to n-tokens
    results = []
    for idx, score in top:
        ntoken = shared_lut.index_to_ntokens.get(idx, f"<idx:{idx}>")
        results.append((ntoken, score))
    
    return results


print("Query function defined")
print()
print("Example queries:")
print('  query("HLLSet")')
print('  query("unified_process")')
print('  query("(reg, zeros)")')

Query function defined

Example queries:
  query("HLLSet")
  query("unified_process")
  query("(reg, zeros)")


In [14]:
# Test queries
test_queries = [
    "HLLSet",
    "unified process",
    "merge",
    "sparse tensor",
]

print("═" * 60)
print("QUERY RESULTS")
print("═" * 60)

for q in test_queries:
    print(f"\nQuery: '{q}'")
    print("-" * 40)
    results = query(q, top_k=5)
    for ntoken, score in results:
        print(f"  {score:.1f}  {ntoken}")

════════════════════════════════════════════════════════════
QUERY RESULTS
════════════════════════════════════════════════════════════

Query: 'HLLSet'
----------------------------------------
  1546.0  {(1, ('n-tokens', 'to')), (2, ('c_line);', '}', 'else')), (2, ('else', '{', 'pyerr_setstring(pyexc_notimplementederror,')), (2, ('and', 'mutual', 'positive')), (2, ('__pyx_l1_error)', '__pyx_gotref((pyobject', '*)__pyx_t_2);')), (2, ('==', 's1.name}', '✓")')), (1, ('values:', 'list[str])')), (2, ('__pyx_decref_typename(type_name);', '__pyx_decref_typename(obj_type_name);', 'return')), (2, ('set.', '"""', 'if')), (1, ('}', 'descr')), (1, ('(unlikely(pymethod_check(__pyx_t_8)))', '{')), (1, ('be', 'a')), (0, ('op->func_code',)), (1, ('contribute)', 'shared_lut')), (1, ('317,', '__pyx_l1_error)')), (2, ('#', '<<<<<<<<<<<<<<', '*')), (2, ('mm->mp_ass_subscript)', '{', 'int')), (1, ('__pyx_pw_4core_8hll_core_7hllcore_1m_1__get__(o);', '}')), (0, ('__pyx_k_c,',)), (1, ('to', 'disambiguate'))

## 11. Interactive Query (with Learning)

In [15]:
def ask(prompt: str, top_k: int = 10, learn: bool = True) -> str:
    """
    Interactive query with feedback loop.
    
    Full sense-process-act-feedback cycle:
    1. Query → HLLSet → HRT → Commit
    2. Find related concepts
    3. Generate response
    4. Response → HLLSet → HRT → Commit (FEEDBACK!)
    
    The manifold learns from BOTH the question AND its own answer.
    
    Args:
        prompt: User query text
        top_k: Number of results to return
        learn: If True, ingest both query AND response (default: True)
    
    Returns:
        Formatted response string from actuator
    """
    global current_hrt, current_W
    
    # SENSE: Process query through prompt perceptron
    new_hrt, new_W, commit, result = prompt_perceptron.process_prompt(
        prompt,
        current_hrt,
        current_W,
        store
    )
    
    if not commit:
        return "No results (empty query)"
    
    if learn:
        # Update state after query ingestion
        current_hrt = new_hrt
        current_W = new_W
    
    # PROCESS: Find related concepts
    query_indices = set()
    if result:
        for edge in result.context_edges:
            query_indices.add(edge.row)
            query_indices.add(edge.col)
    
    # Find reachable (connected concepts)
    AM = Sparse3DMatrix.from_am(current_hrt.am, config)
    layer0 = project_layer(AM, 0)
    reachable = reachable_from(layer0, query_indices, hops=1)
    
    # Convert layer to dict for scoring
    layer0_dict = layer0.to_dict()
    
    # Score by connectivity
    scores = {}
    for idx in reachable:
        if idx in layer0_dict:
            scores[idx] = sum(layer0_dict[idx].values())
    
    # Get top-k, resolve to n-tokens
    top = sorted(scores.items(), key=lambda x: -x[1])[:top_k]
    query_results = []
    for idx, score in top:
        ntoken = shared_lut.index_to_ntokens.get(idx, f"<idx:{idx}>")
        query_results.append((ntoken, score))
    
    # ACT + FEEDBACK: Generate response AND ingest it back
    response_text, final_hrt, final_W = response_actuator.act(
        commit,
        result,
        query_results=query_results,
        hrt=current_hrt,
        W=current_W,
        store=store,
        lut=shared_lut,
        config=config,
        ingest_response=learn,  # Response also gets ingested!
    )
    
    if learn:
        # Update with response-ingested state
        current_hrt = final_hrt
        current_W = final_W
    
    return response_text


def show(response: str) -> None:
    """Print actuator response."""
    print(response)


print("Interactive query with FEEDBACK LOOP:")
print()
print("  response = ask('your query')")
print()
print("What happens:")
print("  1. Query → HLLSet → Commit")
print("  2. Find related concepts")
print("  3. Generate response")
print("  4. Response → HLLSet → Commit  ← FEEDBACK!")
print()
print("The manifold learns from BOTH questions AND its own answers.")

Interactive query with FEEDBACK LOOP:

  response = ask('your query')

What happens:
  1. Query → HLLSet → Commit
  2. Find related concepts
  3. Generate response
  4. Response → HLLSet → Commit  ← FEEDBACK!

The manifold learns from BOTH questions AND its own answers.


## 13. Summary

### Double Loop: Action + Reflection

```text
                    LOOP 1: ACTION
         ┌──────────────────────────────────────┐
         │                                      │
         │  Perceptron ──► Pipeline ──► Actuator ──► OUTPUT
         │  (sense)        (process)    (act)   │
         │                                      │
         └──────────────────────────────────────┘
                                            |
                                          (own output)
                                            │
                    LOOP 2: REFLECTION      ▼
         ┌───────────────────────────────────────┐
         │                                       │
         │  Response   ──►   HLLSet   ──►  Commit ──►  MEMORY
         │  (observe)    (encode)   (record)     │
         │                                       │
         └───────────────────────────────────────┘
```

### Cybernetic Self-Reflection

**Loop 1** is standard sense-process-act.

**Loop 2** is what makes this **cybernetic**:
- The system **observes** its own output
- **Encodes** it as HLLSet (same pipeline as inputs!)
- **Commits** to memory (manifold)

This is **self-reflection** in the cybernetic sense:
> "The system records its own behavior into memory,
>  so future behavior is shaped by past behavior."

### Why Two Commits Per Query

```python
ask("What is HLLSet?")
```

Creates:
1. `prompt_1` → Query committed (system heard you)
2. `response_1` → Response committed (system remembers what it said)

Both shape the manifold. Both influence future responses.

### Memory vs Log

| Traditional Log | Cybernetic Memory |
|-----------------|-------------------|
| External record | Internal state |
| For humans | For system |
| Read-only | Shapes behavior |
| Passive | Active |

The reflection loop isn't just logging - it's the system building a model of its own behavior.

### Emergent Properties

Over time, the double loop creates:

1. **Consistency**: Responses align with past responses
2. **Learning**: Repeated patterns reinforce
3. **Context**: System "remembers" conversation history
4. **Coherence**: Self-model becomes more refined

### The Insight

```text
Traditional AI:  Input → Process → Output
                 (no memory of own outputs)

Fractal Manifold: Input → Process → Output
                              ↓
                         Reflection
                              ↓
                           Memory
                              ↓
                    (shapes next process)
```

**Every output becomes input to future processing.**

This is the cybernetic principle: feedback creates adaptation.

## 14. Hardware Path: FPGA Cybernetic Brain

### Current AI vs Cybernetic AI

| Property | Current AI (LLMs) | Cybernetic AI (Fractal Manifold) |
|----------|-------------------|----------------------------------|
| Memory | Stateless per request | Continuous self-reflection |
| Learning | Offline training only | Online, every interaction |
| Own outputs | Forgotten immediately | Encoded into manifold |
| Hardware | GPUs (matrix multiply) | FPGA (bit operations) |
| Latency | Milliseconds | Microseconds |
| Power | 100-1000W | 1-10W |

### Why FPGA?

Every core operation is hardware-friendly:

```text
┌─────────────────────────────────────────────────────────────────┐
│                    FPGA IMPLEMENTATION                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  OPERATION              FPGA PRIMITIVE         CYCLES           │
│  ─────────              ──────────────         ──────           │
│                                                                 │
│  Hash(token)            Combinational logic    1 cycle          │
│  HLLSet.add()           Leading zero count     1 cycle          │
│  HLLSet.union()         Register OR            1 cycle          │
│  HLLSet.intersect()     Register AND           1 cycle          │
│  Sparse lookup          BRAM read              1-2 cycles       │
│  Edge creation          Parallel pipelines     1 cycle          │
│  Merge                   Tree reduction        log(N) cycles    │
│                                                                 │
│  TOTAL PIPELINE LATENCY: ~10-20 cycles @ 200MHz = 50-100ns      │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Robotic Brain Architecture

```text
                     ┌─────────────────────────────────┐
                     │         FPGA CHIP               │
                     │                                 │
   SENSORS           │  ┌─────────────────────────┐    │           ACTUATORS
   ───────           │  │    PERCEPTRON ARRAY     │    │           ─────────
                     │  │                         │    │
   Camera ──────────►│  │  p_vision   p_audio     │    │◄───────── Servo 1
   Microphone ──────►│  │  p_touch    p_imu       │    │◄───────── Servo 2
   Touch ───────────►│  │  p_lidar    p_prompt    │    │◄───────── Motor
   IMU ─────────────►│  │                         │    │◄───────── Speaker
   LiDAR ───────────►│  └──────────┬──────────────┘    │
                     │             │                   │
                     │             ▼                   │
                     │  ┌─────────────────────────┐    │
                     │  │   UNIFIED PIPELINE      │    │
                     │  │                         │    │
                     │  │  Hash → HLLSet → Merge  │    │
                     │  │  (fully pipelined)      │    │
                     │  │                         │    │
                     │  └──────────┬──────────────┘    │
                     │             │                   │
                     │             ▼                   │
                     │  ┌─────────────────────────┐    │
                     │  │   ACTUATOR ARRAY        │    │
                     │  │                         │    │
                     │  │  a_motor   a_voice      │    │
                     │  │  a_servo   a_display    │    │
                     │  │                         │    │
                     │  └──────────┬──────────────┘    │
                     │             │                   │
                     │             │ REFLECTION LOOP   │
                     │             ▼                   │
                     │  ┌─────────────────────────┐    │
                     │  │   MANIFOLD MEMORY       │    │
                     │  │                         │    │
                     │  │  HBM / DDR / BRAM       │    │
                     │  │  (sparse HRT storage)   │    │
                     │  │                         │    │
                     │  └─────────────────────────┘    │
                     │                                 │
                     └─────────────────────────────────┘
```

### Key FPGA Advantages

1. **Real-time**: Full loop in ~100ns (vs ~100ms for software)
2. **Low power**: 5-10W total (vs 100-1000W for GPU)
3. **Deterministic**: Fixed latency, no GC pauses, no OS jitter
4. **Parallel**: All perceptrons process simultaneously
5. **Embedded**: Runs on battery, fits in robot

### Memory Hierarchy

```text
BRAM (on-chip)     →  Hot n-tokens, LUT cache         ~2MB
HBM (on-package)   →  Active HRT layers               ~4-16GB  
DDR (off-chip)     →  Full manifold, commit history   ~64GB+
```

### Comparison to Neural Networks

| Aspect | Neural Network | Fractal Manifold |
|--------|----------------|------------------|
| Core op | MAC (multiply-add) | Bit ops (AND/OR/XOR) |
| Precision | FP16/FP32/INT8 | 1-bit (HLL registers) |
| Memory | Dense weights | Sparse edges |
| Training | Backprop (offline) | Merge (online) |
| Inference | Forward pass | Graph traversal |
| Interpretable | No (black box) | Yes (edges = relationships) |

### The Vision

A robot with Fractal Manifold FPGA brain:

- **Sees** (camera perceptron) → encodes scene as HLLSet
- **Hears** (audio perceptron) → encodes speech as HLLSet  
- **Responds** (motor actuator) → action selected from manifold
- **Reflects** (feedback loop) → own actions encoded into memory
- **Learns** continuously from every interaction
- **Never forgets** (CRDT merges are permanent)
- **Runs on battery** (5W total power)

This is **Cybernetic AI** - not pattern matching, but genuine sense-process-act-reflect in hardware.